In [3]:
# [BLOCK 1: Setup and Data Loading]
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_curve, confusion_matrix,
    classification_report
)

# 1. Setup MLflow
TRACKING_URI = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_URI)
EXPERIMENT_NAME = "creditcardfraud-mlflow-lifecycle-v3"
mlflow.set_experiment(EXPERIMENT_NAME)

# 2. Load Data
# Assuming creditcard.csv is in the local directory
df = pd.read_csv("creditcard.csv")

# 3. Split Data (Stratified because fraud is rare)
target_col = "Class"
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Preprocessing Pipeline
num_scale_cols = ["Time", "Amount"]
preprocess = ColumnTransformer(
    transformers=[("scale", StandardScaler(), num_scale_cols)],
    remainder="passthrough"
)

print(f"Experiment '{EXPERIMENT_NAME}' is active.")
print(f"Training Data Shape: {X_train.shape}")

Experiment 'creditcardfraud--mlflow-lifecycle-v3' is active.
Training Data Shape: (227845, 30)


In [4]:
# [BLOCK 2: Training Loop - SAFE VERSION]
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, precision_recall_curve

# 1. SAFETY FIX: Create the directory explicitly
if not os.path.exists("../assignment01/artifacts"):
    os.makedirs("../assignment01/artifacts")
    print("Created 'artifacts' directory.")

# Define the grid
run_grid = [
    {"n_estimators": 100, "max_depth": 5,  "min_samples_split": 2, "min_samples_leaf": 1, "class_weight": "balanced"},
    {"n_estimators": 200, "max_depth": 10, "min_samples_split": 5, "min_samples_leaf": 2, "class_weight": "balanced"},
    {"n_estimators": 300, "max_depth": 8,  "min_samples_split": 10, "min_samples_leaf": 4, "class_weight": "balanced"},
    {"n_estimators": 400, "max_depth": 20, "min_samples_split": 2, "min_samples_leaf": 1, "class_weight": "balanced"},
    {"n_estimators": 200, "max_depth": 6,  "min_samples_split": 20, "min_samples_leaf": 10, "class_weight": "balanced"},
]

def plot_pr_curve(y_true, y_proba, path):
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    plt.figure(figsize=(6, 4))
    plt.plot(recall, precision, marker='.', label='Random Forest')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.savefig(path)
    plt.close() # Close to save memory

results = []

print("Starting Training Loop...")

for i, cfg in enumerate(run_grid, start=1):
    try:
        run_name = f"rf_run_{i}_depth_{cfg['max_depth']}"
        print(f"--> Processing Run {i}: {run_name}")

        model = RandomForestClassifier(random_state=42, n_jobs=-1, **cfg)
        pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])

        with mlflow.start_run(run_name=run_name) as run:
            # Tags & Params
            mlflow.set_tags({
                "problem_type": "fraud_detection",
                "model_family": "random_forest",
                "run_purpose": "hyperparameter_tuning" if i > 1 else "baseline"
            })
            mlflow.log_params(cfg)
            mlflow.log_param("test_size", 0.2)

            # Train & Predict
            pipe.fit(X_train, y_train)
            y_proba = pipe.predict_proba(X_test)[:, 1]
            y_pred = (y_proba >= 0.5).astype(int)

            # Metrics
            roc_auc = roc_auc_score(y_test, y_proba)
            avg_precision = average_precision_score(y_test, y_proba)

            mlflow.log_metric("roc_auc", roc_auc)
            mlflow.log_metric("pr_auc", avg_precision)

            # Artifacts (The likely crash point)
            # 1. Confusion Matrix
            cm = confusion_matrix(y_test, y_pred)
            plt.figure(figsize=(5, 4))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
            plt.title(f"Confusion Matrix (Run {i})")
            cm_path = f"artifacts/cm_{run.info.run_id}.png"
            plt.savefig(cm_path)
            plt.close() # Important!
            mlflow.log_artifact(cm_path, artifact_path="plots")

            # 2. PR Curve
            pr_path = f"artifacts/pr_{run.info.run_id}.png"
            plot_pr_curve(y_test, y_proba, pr_path)
            mlflow.log_artifact(pr_path, artifact_path="plots")

            results.append({
                "run_id": run.info.run_id,
                "params": cfg,
                "pr_auc": avg_precision,
                "roc_auc": roc_auc
            })
            print(f"    Finished. PR-AUC: {avg_precision:.4f}")

    except Exception as e:
        print(f"!!! CRASHED ON RUN {i} !!!")
        print(f"Error Message: {e}")
        break # Stop the loop so you can see the error

print("Loop Complete.")

    Finished. PR-AUC: 0.7578
🏃 View run rf_run_5_depth_6 at: http://127.0.0.1:5000/#/experiments/1/runs/87e816de877d4356af7e9cf37b2061c0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
Loop Complete.


In [5]:
# [BLOCK 3: Analysis and Best Run Selection]

# Convert results to DataFrame for easy comparison
compare_df = pd.DataFrame(results)

# Sort by PR-AUC (Average Precision) as it's most important for imbalanced data
compare_df = compare_df.sort_values(by="pr_auc", ascending=False).reset_index(drop=True)

print("Experiment Comparison (Top 3):")
print(compare_df[["run_id", "pr_auc", "roc_auc"]].head(3))

# Select Best Run ID and Params
best_run_id = compare_df.loc[0, "run_id"]
best_params = compare_df.loc[0, "params"]
best_metric = compare_df.loc[0, "pr_auc"]

print(f"\nBest Run ID: {best_run_id} with PR-AUC: {best_metric:.4f}")

Experiment Comparison (Top 3):
                             run_id    pr_auc   roc_auc
0  4aaaabbead584975afd714cd21226315  0.857466  0.959987
1  7b5a3f9548104bd5bf6e02cdfbfb8cc1  0.840541  0.984031
2  f321712bef3146c7a22fd107ae1e6aa6  0.814236  0.985024

Best Run ID: 4aaaabbead584975afd714cd21226315 with PR-AUC: 0.8575


In [6]:
# [BLOCK 4: Retraining and Packaging Best Model]

# We retrain the best config to ensure we have the live object for logging
print(f"Packaging best model with params: {best_params}")

model = RandomForestClassifier(random_state=42, n_jobs=-1, **best_params)
pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
pipe.fit(X_train, y_train)

# Create Input Example and Signature
input_example = X_train.iloc[:5]
y_proba_example = pipe.predict_proba(input_example)[:, 1]
signature = infer_signature(input_example, y_proba_example)

with mlflow.start_run(run_name="best_model_packaged") as run:
    mlflow.set_tag("run_purpose", "packaging_best_model")
    mlflow.log_params(best_params)

    # --- NEW: Calculate and Log Metrics for this specific run ---
    # This ensures the metrics appear in the "best_model_packaged" row
    y_proba_packaged = pipe.predict_proba(X_test)[:, 1]
    roc_auc_packaged = roc_auc_score(y_test, y_proba_packaged)
    pr_auc_packaged = average_precision_score(y_test, y_proba_packaged)

    mlflow.log_metric("roc_auc", roc_auc_packaged)
    mlflow.log_metric("pr_auc", pr_auc_packaged)
    # ------------------------------------------------------------

    # Log the Model with Signature and Input Example
    mlflow.sklearn.log_model(
        sk_model=pipe,
        artifact_path="fraud_model",
        signature=signature,
        input_example=input_example
    )

    packaged_run_id = run.info.run_id
    print(f"Best model packaged in Run ID: {packaged_run_id}")
    print(f"Metrics propagated -> PR-AUC: {pr_auc_packaged:.4f}, ROC-AUC: {roc_auc_packaged:.4f}")

Best model packaged in Run ID: ddcd484fcaea4f9fa1ee6800ab148248
Metrics propagated -> PR-AUC: 0.8575, ROC-AUC: 0.9600
🏃 View run best_model_packaged at: http://127.0.0.1:5000/#/experiments/1/runs/ddcd484fcaea4f9fa1ee6800ab148248
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [7]:
# [BLOCK 5: Model Registry and Lifecycle Management]

client = MlflowClient()
MODEL_REGISTRY_NAME = "CreditCardFraud_RF_Classifier"

# --- Version 1: The Champion (Production) ---
print("Registering Version 1...")
# Register the model (this creates Version 1)
model_v1 = mlflow.register_model(
    model_uri=f"runs:/{packaged_run_id}/fraud_model",
    name=MODEL_REGISTRY_NAME
)
time.sleep(2) # Pause to ensure registration completes

# 1. Transition to Production
client.transition_model_version_stage(
    name=MODEL_REGISTRY_NAME,
    version=model_v1.version,
    stage="Production"
)

# 2. Add Alias, Tag, and Comment
client.set_registered_model_alias(MODEL_REGISTRY_NAME, "champion", model_v1.version)
client.set_model_version_tag(MODEL_REGISTRY_NAME, model_v1.version, "stage", "production")
client.update_model_version(
    name=MODEL_REGISTRY_NAME,
    version=model_v1.version,
    description="Best model version 1"
)


# --- Version 2: The Challenger (Staging) ---
print("Registering Version 2...")
# Register the same model again to simulate a new version/challenger
model_v2 = mlflow.register_model(
    model_uri=f"runs:/{packaged_run_id}/fraud_model",
    name=MODEL_REGISTRY_NAME
)
time.sleep(2)

# 1. Transition to Staging
client.transition_model_version_stage(
    name=MODEL_REGISTRY_NAME,
    version=model_v2.version,
    stage="Staging"
)

# 2. Add Alias, Tag, and Comment
client.set_registered_model_alias(MODEL_REGISTRY_NAME, "challenger", model_v2.version)
client.set_model_version_tag(MODEL_REGISTRY_NAME, model_v2.version, "stage", "staging")
client.update_model_version(
    name=MODEL_REGISTRY_NAME,
    version=model_v2.version,
    description="Best model challenger version 2"
)

print("\n--- Model Registry Status ---")
print(f"Version {model_v1.version}: Alias='champion',   Tag='production', Desc='{model_v1.description}'")
print(f"Version {model_v2.version}: Alias='challenger', Tag='staging',    Desc='{model_v2.description}'")

/var/folders/fk/nsjj191d27x4bdt87fhyv1t00000gn/T/ipykernel_8379/1120417972.py:42: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
